In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import warnings
warnings.filterwarnings("ignore")
import gensim
from tqdm import tqdm
from collections import Counter

In [2]:
# 训练数据merge
df_log = pd.read_csv('train_preliminary/click_log.csv')
# 去除异常数据
df_log = df_log[df_log.click_times < 10]
df_user = pd.read_csv('train_preliminary/user.csv')
# label 从 0 开始
df_user['age'] = df_user['age'].apply(lambda x: x-1)
df_user['gender'] = df_user['gender'].apply(lambda x: x-1)
df_train = pd.merge(df_user, df_log, on='user_id')
df_ad = pd.read_csv('train_preliminary/ad.csv')
df_train = pd.merge(df_train, df_ad, how='left', on='creative_id')
df_train.head()

,user_id,age,gender,time,creative_id,click_times,ad_id,product_id,product_category,advertiser_id,industry
0,1,3,0,43,71691,1,66210,\N,18,14681,326
1,1,3,0,20,821396,1,724607,\N,5,7293,326
2,1,3,0,20,209778,1,188507,136,2,9702,6
3,1,3,0,20,877468,1,773445,\N,5,29455,106
4,1,3,0,64,3068256,1,2638858,1454,2,23575,238


In [34]:
df_train[df_train.industry == r'\N'].head()

,user_id,age,gender,time,creative_id,click_times,ad_id,product_id,product_category,advertiser_id,industry
12,1,3,0,40,122032,1,109959,1334,2,11411,\N
62,3,6,1,14,825434,1,728308,35985,17,14186,\N
64,3,6,1,86,2032927,1,1757244,38250,17,15120,\N
66,3,6,1,86,3581615,1,3083678,37243,17,17151,\N
73,3,6,1,51,2081930,1,1798799,\N,8,15351,\N


In [43]:
df_train[df_train.product_id == '1334'].head()

,user_id,age,gender,time,creative_id,click_times,ad_id,product_id,product_category,advertiser_id,industry
12,1,3,0,40,122032,1,109959,1334,2,11411,\N
103,4,4,0,64,2928770,1,2517968,1334,2,29402,317
480,18,1,1,8,66299,1,61333,1334,2,15494,317
488,18,1,1,4,179839,1,161310,1334,2,15494,317
510,19,2,0,28,1439738,1,1255065,1334,2,21786,317


In [49]:
# product_id使用industry插补
def get_imputation_dict(col, df, target_col):
    ret = {}
    group = df.groupby(col)
    for k, df_v in tqdm(group):
        ret[k] = df_v[target_col].value_counts().index[0]
    return ret

In [50]:
df_imputation = df_train[df_train.product_id != r'\N']
d1 = get_imputation_dict('industry', df_imputation, 'product_id')

100%|██████████| 245/245 [00:10<00:00, 22.97it/s]


In [57]:
df_train[df_train.industry == '64'].head()

,user_id,age,gender,time,creative_id,click_times,ad_id,product_id,product_category,advertiser_id,industry
7,1,3,0,60,2087846,1,1803758,\N,18,44865,64
771,30,3,0,52,1986854,1,1718744,\N,18,44865,64
10149,307,3,1,57,2087846,1,1803758,\N,18,44865,64
16917,516,4,0,51,1987013,1,1718869,\N,18,44865,64
30470,923,3,1,78,2460241,1,2119525,\N,18,44953,64


In [52]:
df_imputation = df_train[df_train.industry != r'\N']
d2 = get_imputation_dict('product_category', df_imputation, 'industry')

100%|██████████| 15/15 [00:13<00:00,  1.14it/s]


In [67]:
df_train['product_id'] = df_train.apply(lambda x: d1.get(x.industry, 0) if x.product_id == r'\N' else x.product_id, axis=1)
df_train['industry'] = df_train.apply(lambda x: d2.get(x.product_category, 0) if x.industry == r'\N' else x.industry, axis=1)


In [68]:
df_train[df_train.product_id == '1334'].head()

,user_id,age,gender,time,creative_id,click_times,ad_id,product_id,product_category,advertiser_id,industry
12,1,3,0,40,122032,1,109959,1334,2,11411,6
103,4,4,0,64,2928770,1,2517968,1334,2,29402,317
480,18,1,1,8,66299,1,61333,1334,2,15494,317
488,18,1,1,4,179839,1,161310,1334,2,15494,317
510,19,2,0,28,1439738,1,1255065,1334,2,21786,317


In [69]:
df_train.head()

,user_id,age,gender,time,creative_id,click_times,ad_id,product_id,product_category,advertiser_id,industry
0,1,3,0,43,71691,1,66210,12829,18,14681,326
1,1,3,0,20,821396,1,724607,12829,5,7293,326
2,1,3,0,20,209778,1,188507,136,2,9702,6
3,1,3,0,20,877468,1,773445,31589,5,29455,106
4,1,3,0,64,3068256,1,2638858,1454,2,23575,238


In [72]:
df_train = df_train.astype(int)

In [73]:
# 测试数据
test_log = pd.read_csv('test/click_log.csv')
test_ad = pd.read_csv('test/ad.csv')
test_log = test_log[test_log.click_times < 10]
df_test = pd.merge(test_log, test_ad, how='left', on='creative_id')
df_test.head()

,time,user_id,creative_id,click_times,ad_id,product_id,product_category,advertiser_id,industry
0,20,3131989,645764,1,573314,58,2,14689,6
1,20,3131989,1027422,1,902764,129,2,42272,6
2,20,3131989,1106443,1,970829,2171,2,37513,322
3,20,3131989,629802,1,559183,\N,18,14678,26
4,59,3131989,2839769,1,2441288,129,2,35328,6


In [75]:
df_test['product_id'] = df_test.apply(lambda x: d1.get(x.industry, 0) if x.product_id == r'\N' else x.product_id, axis=1)
df_test['industry'] = df_test.apply(lambda x: d2.get(x.product_category, 0) if x.industry == r'\N' else x.industry, axis=1)
df_test = df_test.astype(int)

In [76]:
# 去低频词
counter = Counter()
counter.update(df_train.ad_id.values)
counter.update(df_test.ad_id.values)

In [77]:
# 构建词表 去除低频词
ad_id_count = np.array([v for k, v in counter.items()])
np.quantile(ad_id_count, [0.01, 0.1, 0.2, 0.3, 0.5, 0.6, 0.7, 0.8, 0.9])

array([ 1.,  1.,  1.,  1.,  2.,  3.,  5.,  9., 21.])

In [83]:
# 阈值10的creative_id数
(ad_id_count >= 10).sum()

579845

In [86]:
# 构建vocab  0 for pad, 1 for unk
words = [k for k, v in counter.items() if v >= 10]
vocab_ad = dict(zip(words, range(2, len(words) + 2)))
len(vocab_ad)

579845

In [85]:
# 过滤低频词
print(df_train.shape, df_test.shape)
df_train = df_train[df_train.ad_id.isin(vocab)]
df_test = df_test[df_test.ad_id.isin(vocab)]
print(df_train.shape, df_test.shape)

(30081881, 11) (33582799, 9)
(27041830, 11) (30165743, 9)


In [88]:
vocab = {}
for col in ['creative_id', 'product_id', 'product_category', 'advertiser_id', 'industry']:
    words = set(set(df_train[col].values) | set(df_test[col].values))
    vocab[col] = dict(zip(words, range(2, len(words) + 2)))

In [89]:
vocab['ad_id'] = vocab_ad

In [90]:
# 排序
df_train.sort_values(by=['user_id', 'time'], inplace=True)
df_test.sort_values(by=['user_id', 'time'], inplace=True)

In [91]:
df_train.head()

,user_id,age,gender,time,creative_id,click_times,ad_id,product_id,product_category,advertiser_id,industry
1,1,3,0,20,821396,1,724607,12829,5,7293,326
2,1,3,0,20,209778,1,188507,136,2,9702,6
3,1,3,0,20,877468,1,773445,31589,5,29455,106
5,1,3,0,39,1683713,1,1458878,12829,5,14668,326
12,1,3,0,40,122032,1,109959,1334,2,11411,6


In [92]:
df_train[df_train.click_times > 8].head()

,user_id,age,gender,time,creative_id,click_times,ad_id,product_id,product_category,advertiser_id,industry
7688,233,1,0,72,3375481,9,2904869,1261,18,18227,6
197527,5868,2,0,68,2464419,9,2123143,32234,18,14681,295
269755,8028,3,1,44,550614,9,489804,161,2,16582,231
291087,8634,6,0,19,11050,9,12486,1261,18,8385,6
405878,12083,7,0,19,168293,9,150849,145,2,37857,60


In [97]:
# 保存 
train_group = df_train.groupby('user_id')
with open('./datas/train.csv', 'w') as f:
    for uid, df_user in tqdm(train_group):
        values = df_user.values
        user = '_'.join([str(uid), str(values[0, 1]), str(values[0, 2])])
        creative_id = '_'.join([str(vocab['creative_id'][x]) for x in values[:, 4]])
        ad_id = '_'.join([str(vocab['ad_id'][x]) for x in values[:, 6]])
        product_id = '_'.join([str(vocab['product_id'][x]) for x in values[:, 7]])
        product_category = '_'.join([str(vocab['product_category'][x]) for x in values[:, 8]])
        advertiser_id = '_'.join([str(vocab['advertiser_id'][x]) for x in values[:, 9]])
        industry = '_'.join([str(vocab['industry'][x]) for x in values[:, 10]])
        line = ' '.join([user, creative_id, ad_id, product_id, product_category, advertiser_id, industry]) + '\n'
        f.write(line)

100%|██████████| 899999/899999 [09:16<00:00, 1616.29it/s]


In [ ]:
df_test.head()

In [98]:
test_group = df_test.groupby('user_id')
with open('./datas/test.csv', 'w') as f:
    for uid, df_user in tqdm(test_group):
        values = df_user.values
        user = str(uid)
        creative_id = '_'.join([str(vocab['creative_id'][x]) for x in values[:, 2]])
        ad_id = '_'.join([str(vocab['ad_id'][x]) for x in values[:, 4]])
        product_id = '_'.join([str(vocab['product_id'][x]) for x in values[:, 5]])
        product_category = '_'.join([str(vocab['product_category'][x]) for x in values[:, 6]])
        advertiser_id = '_'.join([str(vocab['advertiser_id'][x]) for x in values[:, 7]])
        industry = '_'.join([str(vocab['industry'][x]) for x in values[:, 8]])
        line = ' '.join([user, creative_id, ad_id, product_id, product_category, advertiser_id, industry]) + '\n'
        f.write(line)

100%|██████████| 999999/999999 [10:31<00:00, 1584.48it/s] 


In [100]:
# gender 1 , age 3
set(test_log.user_id.unique()) - set(df_test.user_id.unique())

{3086425}

In [101]:
for col in ['creative_id', 'ad_id', 'product_id', 'product_category', 'advertiser_id', 'industry']:
    print(len(vocab[col]) + 2)

757880
579847
11108
20
24366
302


In [3]:
train_datas = []
with open('./datas/train.csv', 'r') as f:
    train_datas = [line.strip().split()[1:] for line in tqdm(f)]

899999it [00:13, 66480.98it/s]


In [4]:
with open('./datas/test.csv', 'r') as f:
    test_datas = [line.strip().split()[1:] for line in tqdm(f)]

999999it [00:17, 58636.41it/s]


In [4]:
creative_id, ad_id, product_id, product_category, advertiser_id, industry = [], [], [], [], [], []
for line in tqdm(train_datas + test_datas):
    creative_id.append(line[0].split('_'))
    ad_id.append(line[1].split('_'))
    product_id.append(line[2].split('_'))
    product_category.append(line[3].split('_'))
    advertiser_id.append(line[4].split('_'))
    industry.append(line[5].split('_'))

100%|██████████| 1899998/1899998 [02:50<00:00, 11148.24it/s]


In [6]:
model_1 = gensim.models.Word2Vec(creative_id, min_count=1, size=256, workers=-1, iter=100)
model_2 = gensim.models.Word2Vec(ad_id, min_count=1, size=256, workers=-1, iter=100)
model_3 = gensim.models.Word2Vec(product_id, min_count=1, size=256, workers=-1, iter=100)
model_4 = gensim.models.Word2Vec(advertiser_id, min_count=1, size=256, workers=-1, iter=100)

In [7]:
model_1.save('./word2vec/c_id.model')
model_2.save('./word2vec/ad_id.model')
model_3.save('./word2vec/p_id.model')
model_4.save('./word2vec/a_id.model')

In [8]:
length = np.array([len(x) for x in creative_id])

In [9]:
np.quantile(length, [0.8, 0.9, 0.95, 0.98, 0.99])

array([ 41.,  59.,  79., 110., 137.])

In [10]:
np.zeros(10)

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [5]:
creative_id, ad_id, product_id, product_category, advertiser_id, industry = [], [], [], [], [], []
for line in tqdm(train_datas + test_datas):
    ad_id.append(line[1].split('_'))

100%|██████████| 1899998/1899998 [00:35<00:00, 53820.56it/s] 


In [6]:
model = gensim.models.Word2Vec(ad_id, min_count=1, size=512, workers=-1, iter=100)

In [7]:
model.save('./word2vec/ad_id_512.model')